<a href="https://colab.research.google.com/github/tanyaryabov/ML/blob/master/RS_colloborative_filtering_model_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive

Mounted at /content/drive
MyDrive  Shareddrives


In [ ]:
#!pip install surprise
from surprise import SVD
from surprise import Dataset, Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
data = Dataset.load_builtin('ml-100k')

     |████████████████████████████████| 11.8MB 297kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617544 sha256=bccd5a4dd3351b8c0a77657055a3d793d4ba3b5070809c27a3f27b85f07f19ae
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise
Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
#Dividing the data in test and validation
trainset,testset= train_test_split(data, test_size=.20)

SVD- Singular Value Decomposition unsupervised ML algorithm. SVD constructs a matrix with the row of users and columns of items and the elements are given by the users’ ratings. SVD decomposes a matrix into three other matrices and extracts the factors from the factorization of a high-level (user-item-rating) matrix.
Matrix U: singular matrix of (user*latent factors)
Matrix S: diagonal matrix (shows the strength of each latent factor)
Matrix U: singular matrix of (item*latent factors)

In [ ]:
#Initializinf SVD
#n_factors- The number of factors. Default is 100
#n_epochs- The number of iteration of the SGD procedure. Default is 20
#lr_all – The learning rate for all parameters. Default is 0.005
model= SVD(n_factors= 300, n_epochs=30, lr_all=0.01,)
#Fitting the model

from surprise.model_selection import cross_validate
cross_validate(model, data, measures=['RMSE'], cv=5, verbose=True)


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9414  0.9555  0.9462  0.9490  0.9390  0.9462  0.0058  
Fit time          19.50   19.42   19.37   19.50   19.44   19.45   0.05    
Test time         0.17    0.18    0.24    0.18    0.20    0.19    0.03    


{'fit_time': (19.498005390167236,
  19.422816276550293,
  19.37059473991394,
  19.50146222114563,
  19.442075729370117),
 'test_rmse': array([0.94136544, 0.95548972, 0.94623723, 0.94900329, 0.93897723]),
 'test_time': (0.17139768600463867,
  0.17644715309143066,
  0.24315404891967773,
  0.1773083209991455,
  0.19768953323364258)}

The MAE is a linear score which means that all the individual differences are weighted equally in the average. The RMSE is a quadratic scoring rule which measures the average magnitude of the error. ... Since the errors are squared before they are averaged, the RMSE gives a relatively high weight to large errors

In [ ]:
from surprise.model_selection import LeaveOneOut
# define a cross-validation iterator
loo= LeaveOneOut(n_splits=5)
model = SVD()

for trainset, testset in loo.split(data):

    # train and test algorithm.
    model.fit(trainset)
    predictions = model.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.9386
RMSE: 0.9418
RMSE: 0.9851
RMSE: 0.9761
RMSE: 0.9422


In [ ]:
model.fit(trainset)
#making predictions
predictions= model.test(testset)
#accuracy Root Mean Squared Error
accuracy.rmse(predictions)

RMSE: 0.9484


0.9484022867956294

In [ ]:
#prediction rating for a user and movie

uid= str(880)
iid= str(476)
#get a prediction for specific users and items:
pred= model.predict(uid, iid)

In [ ]:
#View predicting rating
pred.est

2.9616042136372123

In [ ]:
!pip install d2l
!pip install mxnet
from d2l import mxnet as d2l
from mxnet import gluon, np
import os
import pandas as pd
d2l.DATA_HUB['ml-100k'] = (
    'http://files.grouplens.org/datasets/movielens/ml-100k.zip',
    'cd4dcac4241c8a4ad7badc7ca635da8a69dddb83')

#@save
def read_data_ml100k():
    data_dir = d2l.download_extract('ml-100k')
    names = ['user_id', 'item_id', 'rating', 'timestamp']
    data = pd.read_csv(os.path.join(data_dir, 'u.data'), '\t', names=names,
                       engine='python')
    num_users = data.user_id.unique().shape[0]
    num_items = data.item_id.unique().shape[0]
    return data, num_users, num_items


In [ ]:
data, num_users, num_items = read_data_ml100k()


In [ ]:
data

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156
